# Your First Weather Radar in 60 Seconds

**What if you could access a week of weather radar data—without downloading a single file?**

Below is a NEXRAD radar image capturing the classic **hook echo** signature of an EF2 tornado in Kansas on March 13, 2024. The left panel shows reflectivity with the characteristic hook-shaped precipitation pattern, while the right panel reveals the storm-relative velocity couplet—opposing winds indicating intense rotation.

<img src="../images/nexrad_hook_echo_tornado_kansas_2024.jpg" width=700 alt="NEXRAD radar of an EF2 tornado in Kansas on March 13, 2024, showing hook echo and velocity couplet">

*Source: National Weather Service, Federal Aviation Administration & United States Air Force — [NEXRAD KTWX](https://commons.wikimedia.org/w/index.php?curid=146468564). Public Domain.*

---

Traditional weather radar analysis requires downloading hundreds of individual files, often totaling dozens of gigabytes. But with cloud-native radar archives, you can access **92 GB of radar data in under 5 seconds** by streaming only the metadata you need.

In this tutorial, you'll learn to:
- Connect to a live weather radar archive in the cloud
- Navigate hierarchical radar data structures
- Understand what radar actually measures (using plain English)
- Visualize all 5 polarimetric variables
- Explore version history with Git-like time travel

Let's dive in.

## The 5-Second Challenge

Watch how fast we can connect to **92 GB** of radar data.

No downloads. No file iteration. Just instant access.

```{tip}
The timer below shows *metadata loading time*—not data transfer. The actual radar measurements stream on-demand only when you need them.
```

In [ ]:
# Core libraries
# For geographic context map
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmweather  # noqa: F401 - Radar-specific colormaps
import icechunk as ic  # Cloud-native versioned storage
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
# Connect to NEXRAD KLOT data on Open Storage Network
# This is publicly accessible—no credentials needed!
storage = ic.s3_storage(
    bucket="nexrad-arco",
    prefix="KLOT-RT",
    endpoint_url="https://umn1.osn.mghpcc.org",
    anonymous=True,
    force_path_style=True,
    region="us-east-1",
)

# Open the repository and create a read-only session
repo = ic.Repository.open(storage)
session = repo.readonly_session("main")

print("✓ Connected to repository: nexrad-arco/KLOT-RT")
print("✓ Session opened on branch: main")

In [ ]:
%%time
# Open the entire radar archive (lazy loading)
dtree = xr.open_datatree(
    session.store,
    zarr_format=3,
    consolidated=False,
    chunks={},
    engine="zarr",
)

In [ ]:
# Check the total dataset size
size_gb = dtree.nbytes / 1024**3
print(f"Connected to {size_gb:.1f} GB of radar data")
print("Metadata loaded in ~3-5 seconds")
print("Data streams on-demand (zero download required)")

---

## Where Are We Looking?

Before diving into the data, let's get our bearings. **KLOT** is the radar code for the NEXRAD station near Chicago, Illinois.

```{note}
NEXRAD radars scan up to a **~460 km (~285 mile) radius** for reflectivity. Think of it like a lighthouse beam that rotates 360° while tilting at different angles. Doppler velocity coverage is smaller (~230 km) due to range folding constraints.
```

In [ ]:
# KLOT radar location
klot_lat = 41.6044
klot_lon = -88.0847
coverage_radius_km = 460

# Create map showing radar coverage
fig = plt.figure(figsize=(6, 5))
ax = plt.axes(
    projection=ccrs.LambertConformal(
        central_longitude=klot_lon, central_latitude=klot_lat
    )
)

# Set extent: ~500 km around radar
ax.set_extent(
    [klot_lon - 5.5, klot_lon + 5.5, klot_lat - 4.5, klot_lat + 4.5],
    crs=ccrs.PlateCarree(),
)

# Add geographic features
ax.add_feature(cfeature.STATES, linewidth=1.5, edgecolor="black")
ax.add_feature(cfeature.COASTLINE, linewidth=1)
ax.add_feature(cfeature.LAKES, alpha=0.5, facecolor="lightblue")

# Draw coverage radius using geodesic circle (proper projection)
theta = np.linspace(0, 2 * np.pi, 100)
# Calculate circle points in lat/lon then transform
circle_lons = klot_lon + (coverage_radius_km / 111) * np.cos(theta) / np.cos(
    np.radians(klot_lat)
)
circle_lats = klot_lat + (coverage_radius_km / 111) * np.sin(theta)
ax.fill(
    circle_lons,
    circle_lats,
    color="red",
    alpha=0.15,
    transform=ccrs.PlateCarree(),
    label="~460 km coverage",
)
ax.plot(
    circle_lons,
    circle_lats,
    color="red",
    linewidth=1.5,
    alpha=0.5,
    transform=ccrs.PlateCarree(),
)

# Mark radar location
ax.plot(
    klot_lon,
    klot_lat,
    marker="*",
    markersize=15,
    color="red",
    transform=ccrs.PlateCarree(),
    label="KLOT Radar",
)

# Add city markers
cities = {"Chicago": (41.8781, -87.6298), "Rockford": (42.2711, -89.0940)}
for city, (lat, lon) in cities.items():
    ax.plot(
        lon, lat, marker="o", markersize=5, color="black", transform=ccrs.PlateCarree()
    )
    ax.text(lon + 0.12, lat, city, fontsize=8, transform=ccrs.PlateCarree(), ha="left")

ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3)
ax.legend(loc="upper right", fontsize=9)
ax.set_title(
    "NEXRAD KLOT Coverage Area\nChicago, Illinois", fontsize=12, fontweight="bold"
)
plt.tight_layout()
plt.show()

---

## Radar 101: What Are We Actually Measuring?

Think of weather radar like a **flashlight in fog**:

1. **Send a pulse**: The radar emits a microwave beam (like turning on your flashlight)
2. **Hit particles**: Raindrops, snowflakes, and bugs scatter the energy back
3. **Listen for echoes**: The radar measures what bounces back and how long it took

### Why Radar Spins and Tilts

To see the entire storm, radar does a **360° rotation** at multiple **elevation angles**:

```
       ↑ 19.5° ─────────────→  (High sweep: sees storm tops)
       ↑ 10.0° ───────────────→  (Mid-level sweep)
       ↑  0.5° ─────────────────→  (Low sweep: sees near ground)
      📡 Radar
```

Each complete rotation at one angle is called a **sweep**. Multiple sweeps at different angles form a **volume scan** (like stacking pancakes to build a 3D picture).

```{tip}
**PPI = Plan Position Indicator**: This is the classic radar view—looking down from above at one elevation angle. It's what meteorologists show on TV!
```

### What Makes Modern Radar "Polarimetric"?

Traditional radar only measured *how much* energy came back. **Dual-polarization radar** (like NEXRAD) is smarter—it sends pulses in two directions (horizontal and vertical) and compares the difference.

This reveals particle **shape and behavior**, helping us distinguish rain from hail, snow, bugs, or even debris from tornadoes.

---

## Understanding the DataTree Structure

The Radar DataTree organizes data hierarchically—like folders on your computer:

```
/
├── VCP-34/              ← Volume Coverage Pattern 34 ("clear air mode")
│   ├── sweep_0/         ← Lowest elevation angle (~0.5°)
│   │   ├── DBZH         ← Reflectivity
│   │   ├── ZDR          ← Differential reflectivity
│   │   ├── RHOHV        ← Correlation coefficient
│   │   └── PHIDP        ← Differential phase
│   ├── sweep_1/         ← Next elevation angle (~1.5°)
│   │   └── VELOCITY     ← Doppler velocity (not available in sweep_0)
│   └── ...
│
├── VCP-212/             ← VCP-212 ("precipitation mode")
│   ├── sweep_0/
│   └── ...
└── ...
```

### What's a VCP?

A **Volume Coverage Pattern (VCP)** is like a recipe that tells the radar:
- How many elevation angles to scan
- How fast to rotate
- What pulse characteristics to use

Common VCPs:
- **VCP-34**: "Clear air mode" (14 sweeps, slower, high sensitivity)
- **VCP-212**: "Precipitation mode" (14 sweeps, faster, optimized for rain)
- **VCP-12**: "Severe weather mode" (14 sweeps, very fast, max time resolution)

```{note}
The radar automatically switches VCPs based on weather conditions. That's why you see multiple VCP folders—they represent different scanning strategies used throughout the day.
```

In [ ]:
# List available Volume Coverage Patterns
print("Available VCPs in this archive:")
for vcp in sorted(dtree.children):
    print(f"  - {vcp}")

In [ ]:
# Explore VCP-34 structure
print("\nSweeps in VCP-34:")
for sweep in sorted(dtree["VCP-34"].children):
    print(f"  - {sweep}")

In [ ]:
# Look inside a single sweep - xarray's beautiful representation
sweep_ds = dtree["VCP-34/sweep_0"].ds
sweep_ds  # xarray displays this beautifully in Jupyter

### The Time Dimension: Your Superpower

Notice the `vcp_time` dimension? This is where cloud-native radar shines.

**Traditional workflow**:
```python
# Painful file iteration
for file in list_of_1000_files:
    download(file)  # Wait... wait... wait...
    process(file)
```

**Cloud-native workflow**:
```python
# Direct time slicing—no loops!
data = dtree['VCP-34/sweep_0'].sel(vcp_time=slice('2025-12-13 14:00', '2025-12-13 16:00'))
```

The data streams only what you need, when you need it.

---

## The Four Key Variables: What Does Radar Actually Tell Us?

Dual-polarization radar measures several key variables. Here are the four available at the lowest sweep:

| Variable | Plain English | What It Reveals | Units |
|----------|---------------|-----------------|-------|
| **DBZH** | "How much stuff is in the air" | Intensity of precipitation (heavier rain = higher values) | dBZ |
| **ZDR** | "Are the drops shaped like pancakes?" | Particle shape (big raindrops flatten; hail is round) | dB |
| **RHOHV** | "How uniform are the targets?" | Purity of precipitation type (0.99+ = pure rain; low = mixed/debris) | 0-1 |
| **PHIDP** | "How much does the wave rotate?" | Cumulative phase shift (used to estimate total rainfall) | degrees |

```{note}
**Doppler velocity (VRADH)** is available in sweep_1 and higher sweeps, showing particle motion toward or away from the radar.
```

```{tip}
**Meteorologist's secret**: When all four variables tell a consistent story (high Z, high ZDR, high ρ), you can confidently say "heavy rain." When they contradict (high Z, low ρ), something weird is happening—maybe hail, or even tornado debris.
```

### Let's See All Four

We'll visualize a single radar scan showing all four variables from a December 2025 winter storm:

In [ ]:
# Select a single timestamp from December 13, 2025
target_time = "2025-12-13 15:36"
scan = dtree["VCP-34/sweep_0"].sel(vcp_time=target_time, method="nearest")

# Check what time we actually got
actual_time = scan.vcp_time.values
print(f"Selected scan: {actual_time}")
print(f"Elevation angle: {scan.sweep_fixed_angle.values:.2f}°")

In [ ]:
# Create 4-panel visualization of polarimetric variables
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.flatten()

# Configuration for each variable (4 from sweep_0)
variables = [
    {
        "var": "DBZH",
        "cmap": "ChaseSpectral",
        "vmin": -10,
        "vmax": 70,
        "label": "dBZ",
        "title": "DBZH: Intensity",
    },
    {
        "var": "ZDR",
        "cmap": "ChaseSpectral",
        "vmin": -2,
        "vmax": 6,
        "label": "dB",
        "title": "ZDR: Drop Shape",
    },
    {
        "var": "RHOHV",
        "cmap": "viridis",
        "vmin": 0.7,
        "vmax": 1.0,
        "label": "unitless",
        "title": "RHOHV: Uniformity",
    },
    {
        "var": "PHIDP",
        "cmap": "twilight_shifted",
        "vmin": 0,
        "vmax": 180,
        "label": "degrees",
        "title": "PHIDP: Phase",
    },
]

for idx, config in enumerate(variables):
    var = config["var"]
    scan[var].plot(
        ax=axes[idx],
        x="x",
        y="y",
        cmap=config["cmap"],
        vmin=config["vmin"],
        vmax=config["vmax"],
        add_colorbar=True,
        cbar_kwargs={"label": config["label"], "shrink": 0.8},
    )
    axes[idx].set_title(config["title"], fontsize=11, fontweight="bold")
    axes[idx].set_xlabel("East (m)", fontsize=9)
    axes[idx].set_ylabel("North (m)", fontsize=9)

fig.suptitle(
    f"Polarimetric Variables - KLOT {str(actual_time)[:19]} UTC",
    fontsize=12,
    fontweight="bold",
    y=0.98,
)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

```{tip}
**Interpretation tip**: Don't interpret variables in isolation! High DBZH could be rain *or* hail—you need ZDR and RHOHV together to distinguish them. When all four variables tell a consistent story, you can confidently identify precipitation type.
```

---

## Time-Based Selection: The Killer Feature

Traditional radar analysis requires looping through individual files. With the DataTree, you select by **time** like you're browsing a calendar.

### Select a Single Scan

Use `method="nearest"` to find the closest scan to your target time:

In [ ]:
# What was happening at 3:30 PM on December 13?
afternoon_scan = dtree["VCP-34/sweep_0"].sel(
    vcp_time="2025-12-13 15:30", method="nearest"
)

print("Requested: 2025-12-13 15:30 UTC")
print(f"Actual scan: {afternoon_scan.vcp_time.values}")
print(f"Variables: {list(afternoon_scan.data_vars)}")

### Select a Time Range

Want to analyze 2 hours of data? Use `slice()`:

In [ ]:
# Get all scans from 2 PM to 4 PM
two_hours = dtree["VCP-34/sweep_0"].sel(
    vcp_time=slice("2025-12-13 14:00", "2025-12-13 16:00")
)

num_scans = len(two_hours.vcp_time)
print(f"Number of scans in 2-hour window: {num_scans}")
print(f"First: {two_hours.vcp_time.values[0]}")
print(f"Last:  {two_hours.vcp_time.values[-1]}")
print(f"Data volume: {two_hours.nbytes / 1024**3:.2f} GB (lazy - not downloaded yet)")

```{important}
**Lazy Evaluation Magic**

When you run the cell above, **no data was downloaded**. You only loaded metadata (timestamps, dimensions, coordinates).

The actual measurements stream on-demand when you:
- Call `.plot()` to visualize
- Call `.compute()` to load into memory
- Perform calculations that require values

This means you can set up complex analyses (filtering, averaging, computations) before committing to any data transfer.
```

---

## Watching Storm Evolution

Let's visualize how reflectivity changed over a 2-hour period. We'll create a time series showing storm development:

In [ ]:
# Select a 2-hour window and sample every 5th scan
time_series = dtree["VCP-34/sweep_0"].sel(
    vcp_time=slice("2025-12-13 14:00", "2025-12-13 16:00")
)
sample_indices = np.arange(0, len(time_series.vcp_time), 5)
sampled_data = time_series.isel(vcp_time=sample_indices)

# Create multi-panel plot showing storm evolution
num_times = min(len(sampled_data.vcp_time), 6)
fig, axes = plt.subplots(1, 3, figsize=(12, 3), sharey=True)
axes = axes.flatten()

for idx, time_val in enumerate(sampled_data.vcp_time.values[:num_times]):
    scan = sampled_data.sel(vcp_time=time_val)
    scan.DBZH.plot(
        ax=axes[idx],
        x="x",
        y="y",
        cmap="ChaseSpectral",
        vmin=-10,
        vmax=70,
        add_colorbar=False,
    )
    time_str = str(time_val)[:16].replace("T", " ")
    axes[idx].set_title(f"{time_str} UTC", fontsize=10, fontweight="bold")
    axes[idx].set_xlabel("East (m)", fontsize=9)
    axes[idx].set_ylabel("North (m)", fontsize=9)

fig.colorbar(
    axes[0].collections[0], ax=axes, label="Reflectivity (dBZ)", shrink=0.8, pad=0.02
)
fig.suptitle(
    "Storm Evolution: NEXRAD KLOT (December 13, 2025)",
    fontsize=12,
    fontweight="bold",
    y=1.00,
)
plt.show()

---

## Time Travel with Icechunk: Git for Radar Data

Remember when we connected to the radar archive using `icechunk`? That wasn't just a storage layer—it's a **version control system** for scientific data.

Think of it like Git, but for multi-dimensional arrays:
- Every data update creates a new **snapshot** (commit)
- You can view the **commit history** (`ancestry()`)
- You can **time travel** to any previous version
- Changes are **ACID-compliant** (atomic, consistent, isolated, durable)

### Why This Matters for Science

```{note}
**Reproducibility crisis solved**: With Icechunk, you can reference an exact data snapshot in your paper. Future researchers can load *exactly* the same data you used—even if the archive has been updated since publication.
```

### View the Commit History

In [ ]:
# View the last 3 commits (snapshots) to the radar archive
for i, snapshot in enumerate(repo.ancestry(branch="main")):
    print(f"#{i}: {snapshot.id}")
    print(f"    Date: {snapshot.written_at.strftime('%Y-%m-%d %H:%M:%S UTC')}")
    print(f"    Msg:  {snapshot.message[:80] if snapshot.message else 'No message'}")
    print()
    if i >= 3:  # Show last 10 snapshots
        print("... (more snapshots)")
        break

### Time Travel for Reproducibility

Each snapshot ID represents the **exact state** of the data at that moment. This enables:

- **Reproducible science**: Reference a specific snapshot in your paper—future researchers can load *exactly* the same data
- **Data provenance**: Track how the archive evolved over time
- **Debugging**: Compare current data to historical versions if something seems off

```{tip}
**For publications**: Include the snapshot ID in your methods section. Anyone can reproduce your exact analysis by loading that specific commit, even years later.
```

---

## Summary: What You've Learned

In this tutorial, you:

- **Connected** to 92 GB of radar data (in ~3-5 seconds)
- **Understood** how radar works (it's a flashlight in fog!)
- **Navigated** a hierarchical VCP → sweep → variable structure
- **Visualized** four polarimetric variables (DBZH, ZDR, RHOHV, PHIDP)
- **Selected** data by time (no file iteration needed)
- **Explored** version history like Git for data

### Key Takeaways

1. **Cloud-native = speed**: Metadata loads instantly; data streams on-demand
2. **Time is your friend**: Select by timestamp instead of looping through files
3. **Four variables tell the story**: Use all of them together to understand precipitation
4. **Reproducibility built-in**: Icechunk snapshots ensure your analysis is forever reproducible

---

## Next Steps: Level Up Your Skills

Ready for more advanced analysis? Try these notebooks:

### **[2. QVP Workflow Comparison](2.QVP-Workflow-Comparison)**
Learn about **Quasi-Vertical Profiles (QVP)** for precipitation analysis:
- See the **36x speedup** over traditional file-based workflows
- Reproduce a published scientific figure from Ryzhkov et al. (2016)

### **[3. QPE Snow Storm](3.QPE-Snow-Storm)**
Apply your skills to a real weather event:
- Compute snow accumulation during the December 2025 Illinois storm
- Use Z-R relationships for quantitative precipitation estimation
- Create geographic accumulation maps

```{admonition} Challenge Yourself
:class: tip

1. **Find the strongest echo**: What's the maximum DBZH value on December 13? At what time and location?
2. **Compare VCPs**: How do sweep angles differ between VCP-34 and VCP-212?
3. **Detect rotation**: Use VELOCITY to identify areas with rotation (opposing velocities)
```

---

## Citation

If you use this data or framework in your research, please cite:

> Ladino-Rincón, A., & Nesbitt, S. W. (2025). *Radar DataTree: A FAIR and Cloud-Native Framework for Scalable Weather Radar Archives.* arXiv:2510.24943. [doi:10.48550/arXiv.2510.24943](https://doi.org/10.48550/arXiv.2510.24943)

---

*Tutorial created by the Radar DataTree team*